In [ ]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
#from datetime import datetime, date, timedelta
from selenium.webdriver.chrome.options import Options
import requests
import json

from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv(".env")

#functions-framework --target=syuzaibetu_hallnavi_scraping --signature-type=event --debug --port=8080
#functions-framework --target=syuzaibetu_hallnavi_scraping --port=8081 --signature-type=event
print('ライブラリの読み込み完了')
# def syuzaibetu_hallnavi_scraping(event,context):
#     global furture_syuzai_list_df_1,cnx

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def insert_data_bulk(df,cnx):
    insert_sql = """INSERT INTO mysql_database.syuzai_report_table (都道府県, 日付, 曜日, 店舗名, 取材名, 媒体名, ランク) values (%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


def insert_data_bulk(df,cnx):
    insert_sql = """INSERT INTO hallnavi_gcp (都道府県, 日付, 曜日, 店舗名, 取材名, 媒体名, ランク) 
    values (%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")

def login_scraping_site(area_name):
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--no-sandbox")

    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install() 
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)

    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))

    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))

    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser


furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','日付','店舗名','取材名','媒体名','ランク'])
browser = login_scraping_site('chubu')
elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
i = 0
while True:
    browser.find_element(By.CLASS_NAME,"head_change_main").click()
    browser.implicitly_wait(10)
    if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
        browser = login_scraping_site('chubu')
    else:
        pass
    try:
        elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
        baitai_name = elements[i].text.split(' ')[0]
        baitai_name = baitai_name.replace('パチマガスロマガじゃ…','パチマガスロマガ').replace('パチンコ店長のホール…','パチンコ店長のホール攻略')

        if  ('県' in baitai_name) or ('府' in baitai_name):
            print('県がついているためパスを処理しました',baitai_name)
            i += 1
            continue

        print(baitai_name)
        elements[i].click()
        if 'ホールナビ' in baitai_name:
            print(baitai_name)
            break
        while True:
            for syuzai_tenpo_data in browser.find_elements(By.CLASS_NAME,"osbox"):
                tenpo_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslh2").text
                #print(tenpo_name.text)
                syuzai_date = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslmd").text
                rank_and_syuzai_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"list_event_name").text
                syuzai_rank = rank_and_syuzai_name.split('\n')[0]
                syuzai_name = rank_and_syuzai_name.split('\n')[1]
                prefectures = syuzai_tenpo_data.find_elements(By.CLASS_NAME,"oslha")[0].text
                #print(baitai_name,syuzai_date ,syuzai_rank,syuzai_name,prefectures)#prefectures
                record = pd.Series([prefectures,syuzai_date, tenpo_name,syuzai_name,baitai_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
                #print(record)
                #break
                browser.implicitly_wait(10)
            if browser.find_element(By.CLASS_NAME,'navi_1_next').text == '次へ':
                browser.find_element(By.CLASS_NAME,'navi_1_next').click()
            else:
                break
        i += 1
        # time.sleep(1)
        #break
        #break
    except Exception as e:
        print('エラー理由',e)
        break
    # if i > 13:
    #     break

#browser.quit()


In [32]:
browser.find_element(By.CLASS_NAME,'navi_1_next').text

'次へ'

In [34]:
pattern = '東京都|北海道|(京都|大阪)府|.{2,3}県'
# 都道府県を抽出する
furture_syuzai_list_df = furture_syuzai_list_df[furture_syuzai_list_df['都道府県'] != '']
# furture_syuzai_list_df['都道府県']=furture_syuzai_list_df['都道府県'].apply(lambda x:re.match(pattern,x).group())
prefectures_list = []
for adress in furture_syuzai_list_df['都道府県']:
    adress = adress.split(']')[-1]
    try:
        #print(re.match(pattern,adress).group())
        prefectures_list.append(re.match(pattern,adress).group())
    except:
        prefectures_list.append('情報なし')
furture_syuzai_list_df['都道府県'] = prefectures_list
furture_syuzai_list_df_1 = furture_syuzai_list_df
furture_syuzai_list_df_1 = pd.concat([furture_syuzai_list_df_1, furture_syuzai_list_df_1['日付'].str.split('(', expand=True)], axis=1).drop('日付', axis=1)
furture_syuzai_list_df_1.rename(columns={0: '日付', 1: '曜日'}, inplace=True)
furture_syuzai_list_df_1['曜日'] = furture_syuzai_list_df_1['曜日'].map(lambda x:x.replace(')',''))
furture_syuzai_list_df_1['日付'] = pd.to_datetime(furture_syuzai_list_df_1['日付'])
furture_syuzai_list_df_1 = furture_syuzai_list_df_1 [['都道府県','日付','曜日',	'店舗名','取材名','媒体名','ランク']]
furture_syuzai_list_df_1

,都道府県,日付,曜日,店舗名,取材名,媒体名,ランク
0,長野県,2023-03-11,土,ニューアサヒ長野青木島店,APチャンネル来店,APチャンネル,C
1,岐阜県,2023-03-11,土,プレイランドキャッスル大垣,APチャンネル来店,APチャンネル,C
2,長野県,2023-03-11,土,ニューアサヒ諏訪高島店,トレジャーハント【numberｓ】,APチャンネル,・
3,長野県,2023-03-12,日,ニューアサヒ茅野店,luckynumber,APチャンネル,・
4,新潟県,2023-03-12,日,DAMZ両津店,ガチレポスリーカード(紫),APチャンネル,C
...,...,...,...,...,...,...,...
1230,岐阜県,2023-03-12,日,プレイランドキャッスル大垣,ホールサーチマン 金枠,ホールサーチマン,・
1231,山梨県,2023-03-15,水,マルハン南アルプス店,ホールサーチマン 金枠,ホールサーチマン,・
1232,山梨県,2023-03-17,金,マルハン南アルプス店,ホールサーチマン 赤枠(パチ),ホールサーチマン,・
1233,愛知県,2023-03-18,土,コンコルド880愛西日比野駅前店,ホールサーチマン 金枠,ホールサーチマン,・


In [ ]:


#### Create dataframe from resultant table ####
cnx = mysql.connector.connect(
                        user = os.getenv('DB_USER_NAME'),
                        password=os.getenv('DB_PASSWORD'), 
                        host=os.getenv('DB_HOST'), 
                        port='3306',
                        database=os.getenv('DB_NAME'))

cursor = cnx.cursor()
sql = "SELECT * FROM syuzai_report_table"
cursor.execute(sql)
#cols = [col[0] for col in cursor.description]
sql_syuzai_report_all_df = pd.DataFrame(cursor.fetchall(),columns = ['都道府県','日付','曜日','店舗名','取材名','媒体名','ランク'])
#df
# cursor = cnx.cursor()
# # #### Connetion Established ####
# # #### Execute query ####
# query1 = ("select * from hallnavi_gcp")
# cursor.execute(query1)
# #### Create dataframe from resultant table ####

sql_syuzai_report_all_df

concat_df = pd.concat([sql_syuzai_report_all_df,furture_syuzai_list_df_1])
concat_df['日付'] = pd.to_datetime(concat_df['日付'])
concat_df = concat_df[~concat_df.duplicated(keep=False)]

concat_df 

In [34]:
insert_data_bulk(concat_df,cnx)
cnx.commit()

post_line_text(f'{len(concat_df)}件の東海の取材予定追加おわり',os.getenv('LINE_TOKEN'))


Insert bulk data
